# Neural network Tuning

In [1]:
import numpy
import pandas as pd
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.wrappers.scikit_learn import KerasClassifier

import warnings
warnings.filterwarnings('ignore')

# part 1 - Tuning batch and epoch

In [2]:
# load the dataset for India diabetes
data = pd.read_csv('diabetes.csv')

In [3]:
#viewing data
data.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [4]:
# split into predictors and response
X = data.iloc[:,0:8] #predictors
Y = data.iloc[:,8] #response

In [5]:
# Function to create model,for KerasClassifier
def create_my_model():
    #defining my model
    mymodel = Sequential()
    mymodel.add(Dense(12, input_dim = 8, activation = 'relu'))
    mymodel.add(Dense(1, activation = 'sigmoid'))
    
    # Compile the model
    mymodel.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
    return mymodel

In [6]:
# create model
model = KerasClassifier(build_fn = create_my_model)

In [7]:
# define the grid search parameters
batchSize = [10, 20, 40, 60, 80, 100]
epochs = [10, 30, 50]

In [8]:
parameter_grid = dict(batch_size=batchSize, epochs=epochs)

mygrid = GridSearchCV(estimator=model, param_grid=parameter_grid, n_jobs=-1, cv=3)
grid_result = mygrid.fit(X, Y)

Epoch 1/30
Epoch 1/10
Epoch 1/10
Epoch 1/30
Epoch 1/50
Epoch 1/30
Epoch 1/10
52/52 [==============================] - 0s 2ms/step - loss: 10.8778 - accuracy: 0.4727
Epoch 2/30
52/52 [==============================] - 0s 1ms/step - loss: 5.5041 - accuracy: 0.4766
Epoch 3/30
52/52 [==============================] - 0s 2ms/step - loss: 4.5205 - accuracy: 0.4961
Epoch 4/30
52/52 [==============================] - 0s 2ms/step - loss: 11.9235 - accuracy: 0.4512
Epoch 2/10
52/52 [==============================] - 0s 1ms/step - loss: 14.4749 - accuracy: 0.3828
Epoch 2/30
52/52 [==============================] - 0s 1ms/step - loss: 4.7792 - accuracy: 0.6016
Epoch 2/30
52/52 [==============================] - 0s 2ms/step - loss: 7.6034 - accuracy: 0.5391
Epoch 2/10
52/52 [==============================] - 0s 1ms/step - loss: 17.7222 - accuracy: 0.4043
Epoch 2/50
52/52 [==============================] - 0s 2ms/step - loss: 12.9198 - accuracy: 0.6367
Epoch 2/10
52/52 [=============================

In [9]:
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

Best: 0.670573 using {'batch_size': 10, 'epochs': 30}


# part 2- Optimizing Activation Function

In [10]:
# Function to create model
def create_my_model(optimizer = 'adam'):
    # create model
    mymodel = Sequential()
    mymodel.add(Dense(12, input_dim = 8, activation = 'relu'))
    mymodel.add(Dense(1, activation = 'sigmoid'))
    # Compile model
    mymodel.compile(loss = 'binary_crossentropy', optimizer = optimizer, metrics = ['accuracy'])
    return mymodel

In [11]:
# create model
model = KerasClassifier(build_fn = create_my_model, epochs = 50, batch_size = 10)

In [12]:
# define the grid search parameters
optimizer = ['SGD','Adadelta', 'RMSprop', 'Adagrad','Adam']
parameter_grid = dict(optimizer=optimizer)

In [13]:
grid = GridSearchCV(estimator = model, param_grid = parameter_grid, n_jobs = -1, cv = 3)
grid_result = grid.fit(X, Y)

Epoch 1/50
Epoch 1/50
Epoch 1/50
Epoch 1/50
Epoch 1/50
Epoch 1/50
Epoch 1/50
Epoch 1/50
52/52 [==============================] - 0s 2ms/step - loss: 2.6090 - accuracy: 0.6230
Epoch 2/50
52/52 [==============================] - 0s 2ms/step - loss: 5.2575 - accuracy: 0.6191
Epoch 2/50
52/52 [==============================] - 0s 1ms/step - loss: 0.6608 - accuracy: 0.6738
Epoch 3/50
52/52 [==============================] - 0s 1ms/step - loss: 7.0937 - accuracy: 0.4453
Epoch 3/50
52/52 [==============================] - 0s 2ms/step - loss: 12.8902 - accuracy: 0.5137
Epoch 2/50
52/52 [==============================] - 0s 819us/step - loss: 3.5548 - accuracy: 0.5703
Epoch 2/50
52/52 [==============================] - 0s 1ms/step - loss: 0.6724 - accuracy: 0.6406
Epoch 3/50
52/52 [==============================] - 0s 1ms/step - loss: 0.6862 - accuracy: 0.6543
Epoch 3/50
52/52 [==============================] - 0s 1ms/step - loss: 21.2420 - accuracy: 0.5625
Epoch 2/50
52/52 [===================

In [14]:
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

Best: 0.660156 using {'optimizer': 'Adam'}


# How can you improve accuracy of the model?

In [15]:
def create_my_model(optimizer = 'Adam', dropout_rate = 0.5):
    model = Sequential()
    model.add(Dense(12, input_dim = 8, activation = 'relu'))
    model.add(Dropout(dropout_rate))
    model.add(Dense(1, activation = 'sigmoid'))
    model.compile(loss = 'binary_crossentropy', optimizer = optimizer, metrics = ['accuracy'])
    return model

In [16]:
model = KerasClassifier(build_fn = create_my_model, epochs = 50, batch_size = 10)

In [17]:
optimizer = ['SGD','Adadelta', 'RMSprop', 'Adagrad','Adam']
dropout_rate = [0.0, 0.2, 0.5]
parameter_grid = dict(optimizer = optimizer, dropout_rate = dropout_rate)

In [18]:
grid = GridSearchCV(estimator = model, param_grid = parameter_grid, n_jobs = -1, cv = 3)
grid_result = grid.fit(X, Y)

Epoch 1/50
Epoch 1/50
Epoch 1/50
Epoch 1/50
Epoch 1/50
Epoch 1/50
Epoch 1/50
Epoch 1/50
52/52 [==============================] - 0s 1ms/step - loss: 2.7103 - accuracy: 0.6328
Epoch 2/50
52/52 [==============================] - 0s 2ms/step - loss: 1.7836 - accuracy: 0.6426
Epoch 2/50
52/52 [==============================] - 0s 1ms/step - loss: 14.8808 - accuracy: 0.5781
Epoch 2/50
52/52 [==============================] - 0s 715us/step - loss: 0.6437 - accuracy: 0.6387
Epoch 3/50
52/52 [==============================] - 0s 2ms/step - loss: 2.4034 - accuracy: 0.6309
Epoch 2/50
52/52 [==============================] - 0s 2ms/step - loss: 18.7717 - accuracy: 0.5098
Epoch 2/50
52/52 [==============================] - 0s 749us/step - loss: 0.6713 - accuracy: 0.6816
Epoch 3/50
52/52 [==============================] - 0s 1ms/step - loss: 52.3535 - accuracy: 0.3613
Epoch 2/50
52/52 [==============================] - 0s 711us/step - loss: 0.6331 - accuracy: 0.6484
Epoch 4/50
Epoch 3/50
52/52 [===

In [19]:
# Summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

Best: 0.687500 using {'dropout_rate': 0.2, 'optimizer': 'Adam'}
